In [1]:
# This Code is written by Muhammad Ahsan Awais
# PhD research scholar at Dublin City University, Ireland.
# Dated 29-April-2023

import mne
import os
import numpy as np
import pandas as pd

In [2]:
P_Num = 1 # Participant ID

In [3]:
# This function deals with all tasks involving RSVP
# Standard RSVP: X1 , X2
# Artifacts + RSVP: X4, X6, X8

def process_file(edf_file_path, csv_file_path):
    raw=mne.io.read_raw_edf(edf_file_path, preload=True)
    # Resampling data from 1000sps to 100sps
    raw = raw.resample(sfreq=100)
    
    #Applying lowpass filter at 30Hz
    raw = raw.filter(.1, 30, method='iir')
    
    #Re-Referencing the data to common average reference (CAR)
    raw= raw.set_eeg_reference(ref_channels='average')
    
    data = raw.get_data()
    Samples=len(data[0])

    annt=mne.events_from_annotations(raw, event_id='auto', regexp='^(?![Bb][Aa][Dd]|[Ee][Dd][Gg][Ee]).*$', use_rounding=True, chunk_duration=None, verbose=None)
    ann0=annt[0]
    Markers_Raw=ann0[:,0]
    L_annt_r=len(Markers_Raw)
    
    # Total number of epohcs are 360, but we have 360 markers in EEG data which indicates the start and end of the experiment
    if (L_annt_r == 362):
        Markers=Markers_Raw[1:-1] # :- Removing Extra Markers:- 1 from start and 1 from end
    else:
        print("Unknown length")
    L_annt0=len(Markers)
    print("Number of markers: ",L_annt0)

    Zeros = [0] * Samples
    Zero_arr = np.array(Zeros)
    Zero_2d = np.reshape(Zero_arr, (len(Zero_arr), 1))
    Zero_2dt=np.transpose(Zero_2d)

    #Loading Labels for RSVP images from csv file
    csv_mark0 = pd.read_csv(csv_file_path, header=None)
    csv_mark = np.array(csv_mark0)

    zero_mark=Zero_arr
    csv1=0
    for x in Markers:   
        zero_mark[x]=csv_mark[csv1]
        csv1=csv1+1

    tar = np.where(zero_mark == 1)
    std = np.where(zero_mark == 2)

    new_events = []
    for st in std[0]:
        new_events.append([st, 0, 2]) 
    for t in tar[0]:
        new_events.append([t, 0, 1])

    event_dict = {'target':1, 'standard':2}
    epoch = mne.Epochs(raw, new_events, tmin=-0.25, tmax=1.0, event_id=event_dict,
                    preload=True, baseline=(-.25, 0))
    
    return epoch

#---------------------------------------------------------------------------
#---------------------------------------------------------------------------
# This function deals with all tasks that does not involve any RSVP
# Baselines: B0, B1, B2
# Artifacts: X3, X5, X7
#---------------------------------------------------------------------------

def process_file2(edf_file_path):
    raw1=mne.io.read_raw_edf(edf_file_path, preload=True)
    # Resampling data from 1000sps to 100sps
    raw1 = raw1.resample(sfreq=100)
    
    #Applying lowpass filter at 30Hz
    raw1 = raw1.filter(.1, 30, method='iir')
    
    #Re-Referencing the data to common average reference (CAR)
    raw1 = raw1.set_eeg_reference(ref_channels='average')
    
    data = raw1.get_data()

    annt=mne.events_from_annotations(raw1, event_id='auto', regexp='^(?![Bb][Aa][Dd]|[Ee][Dd][Gg][Ee]).*$', use_rounding=True, chunk_duration=None, verbose=None)
    ann0=annt[0]
    Markers_Raw=ann0[:,0]
    L_annt_r=len(Markers_Raw)
    
    # Since there is no RSVP, Total number of markers are 4, 1st and last indicating the start and end (i.e., contains garbage). Useful Data is between middle two markers
    if (L_annt_r == 4):
        Markers=Markers_Raw[1:-1] # :- Removing Extra Markers:- 1 from start and 1 from end
    else:
        print("Unknown length")
        
    print("Markers = ",Markers)
    
    Dur=Markers[1]-Markers[0] # Duration of the task
    
    new_events1=np.array([Markers[0], 0, 1])
    new_events1 = new_events1.reshape(3,1)
    new_events=np.transpose(new_events1)
    event_dict = {'Data':1}
    
    if Dur < 5000: # For Baseline (eye Movement).Its duration is 10 seconds instead of 60 seconds
        epoch = mne.Epochs(raw1, new_events, tmin=0.0, tmax=10.0, event_id=event_dict,
                        preload=True, baseline=None)
        
    elif Dur > 7000: # For Artifact trials without RSVP. each trial is 90 secs
        epoch = mne.Epochs(raw1, new_events, tmin=0.0, tmax=90.0, event_id=event_dict,
                       preload=True, baseline=None)   
        
    else: # For Baseline (eyes open and eyes close).
        epoch = mne.Epochs(raw1, new_events, tmin=0.0, tmax=60.0, event_id=event_dict,
                        preload=True, baseline=None)
    
    return epoch

In [4]:
# Only RSVP Tasks ( X1 and X2 )
# There are two X1s and two X2s in each session

#----------------------------------------Session-1----------------------------------------

# Raw EEG edf file path
edf_path1 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X1-eeg.edf'.format(P_Num, P_Num)

# RSVP image labels file path
csv_path1 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Mark\\P{}-Ss1-X1-mrk.csv'.format(P_Num, P_Num)

#Processing the files to extract Epochs using the defined function 'process_file'
epoch1 = process_file(edf_path1,csv_path1)

# Create a DataFrame from the epoch data
df1 = epoch1.to_data_frame()

# Write the DataFrame to a CSV file
df1.to_csv('P{}-Ss1-Data-X1.csv'.format(P_Num), index=False) 


edf_path2 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X2-eeg.edf'.format(P_Num, P_Num)
csv_path2 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Mark\\P{}-Ss1-X2-mrk.csv'.format(P_Num, P_Num)
epoch2 = process_file(edf_path2,csv_path2)
df2 = epoch2.to_data_frame()
df2.to_csv('P{}-Ss1-Data-X2.csv'.format(P_Num), index=False)

#----------------------------------------Session-2----------------------------------------

edf_path3 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X1-eeg.edf'.format(P_Num, P_Num)
csv_path3 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Mark\\P{}-Ss2-X1-mrk.csv'.format(P_Num, P_Num)
epoch3 = process_file(edf_path3,csv_path3)
df3 = epoch3.to_data_frame()
df3.to_csv('P{}-Ss2-Data-X1.csv'.format(P_Num), index=False)

edf_path4 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X2-eeg.edf'.format(P_Num, P_Num)
csv_path4 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Mark\\P{}-Ss2-X2-mrk.csv'.format(P_Num, P_Num)
epoch4 = process_file(edf_path4,csv_path4)
df4 = epoch4.to_data_frame()
df4.to_csv('P{}-Ss2-Data-X2.csv'.format(P_Num), index=False)

#----------------------------------------Session-3----------------------------------------

edf_path5 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X1-eeg.edf'.format(P_Num, P_Num)
csv_path5 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Mark\\P{}-Ss3-X1-mrk.csv'.format(P_Num, P_Num)
epoch5 = process_file(edf_path5,csv_path5)
df5 = epoch5.to_data_frame()
df5.to_csv('P{}-Ss3-Data-X1.csv'.format(P_Num), index=False)

edf_path6 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X2-eeg.edf'.format(P_Num, P_Num)
csv_path6 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Mark\\P{}-Ss3-X2-mrk.csv'.format(P_Num, P_Num)
epoch6 = process_file(edf_path6,csv_path6)
df6 = epoch6.to_data_frame()
df6.to_csv('P{}-Ss3-Data-X2.csv'.format(P_Num), index=False)

#----------------------------------------Session-4----------------------------------------

edf_path7 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X1-eeg.edf'.format(P_Num, P_Num)
csv_path7 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Mark\\P{}-Ss4-X1-mrk.csv'.format(P_Num, P_Num)
epoch7 = process_file(edf_path7,csv_path7)
df7 = epoch7.to_data_frame()
df7.to_csv('P{}-Ss4-Data-X1.csv'.format(P_Num), index=False)

edf_path8 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X2-eeg.edf'.format(P_Num, P_Num)
csv_path8 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Mark\\P{}-Ss4-X2-mrk.csv'.format(P_Num, P_Num)
epoch8 = process_file(edf_path8,csv_path8)
df8 = epoch8.to_data_frame()
df8.to_csv('P{}-Ss4-Data-X2.csv'.format(P_Num), index=False)


Extracting EDF parameters from C:\Users\muhammad\Desktop\Data A\P1\Session-1\Data\P1-Ss1-X1-eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 108999  =      0.000 ...   108.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 30.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['EventCode 16  EpochE', 'Impedance']
Number of markers:  360
Not setting metadata
Not setting metadata
360 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 360 events and 126 original time points ...
0 bad epochs dropped


C:\Users\muhammad\AppData\Local\Temp\ipykernel_18668\2222423111.py:57: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epoch = mne.Epochs(raw, new_events, tmin=-0.25, tmax=1.0, event_id=event_dict,


Extracting EDF parameters from C:\Users\muhammad\Desktop\Data A\P1\Session-1\Data\P1-Ss1-X2-eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 113999  =      0.000 ...   113.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 30.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['EventCode 16  EpochE', 'Impedance']
Number of markers:  360
Not setting metadata
Not setting metadata
360 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 360 events and 126 original time points ...
0 bad epochs dropped


C:\Users\muhammad\AppData\Local\Temp\ipykernel_18668\2222423111.py:57: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epoch = mne.Epochs(raw, new_events, tmin=-0.25, tmax=1.0, event_id=event_dict,


Extracting EDF parameters from C:\Users\muhammad\Desktop\Data A\P1\Session-2\Data\P1-Ss2-X1-eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 108999  =      0.000 ...   108.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 30.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['EventCode 16  EpochE', 'Impedance']
Number of markers:  360
Not setting metadata
Not setting metadata
360 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 360 events and 126 original time points ...
0 bad epochs dropped


...

C:\Users\muhammad\AppData\Local\Temp\ipykernel_18668\2222423111.py:57: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epoch = mne.Epochs(raw, new_events, tmin=-0.25, tmax=1.0, event_id=event_dict,


Extracting EDF parameters from C:\Users\muhammad\Desktop\Data A\P1\Session-4\Data\P1-Ss4-X1-eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 111999  =      0.000 ...   111.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 30.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['EventCode 16  EpochE', 'Impedance']
Number of markers:  360
Not setting metadata
Not setting metadata
360 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 360 events and 126 original time points ...
0 bad epochs dropped


C:\Users\muhammad\AppData\Local\Temp\ipykernel_18668\2222423111.py:57: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epoch = mne.Epochs(raw, new_events, tmin=-0.25, tmax=1.0, event_id=event_dict,


Extracting EDF parameters from C:\Users\muhammad\Desktop\Data A\P1\Session-4\Data\P1-Ss4-X2-eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 109999  =      0.000 ...   109.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 30.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['EventCode 16  EpochE', 'Impedance']
Number of markers:  360
Not setting metadata
Not setting metadata
360 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 360 events and 126 original time points ...
0 bad epochs dropped


C:\Users\muhammad\AppData\Local\Temp\ipykernel_18668\2222423111.py:57: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epoch = mne.Epochs(raw, new_events, tmin=-0.25, tmax=1.0, event_id=event_dict,


In [5]:
# RSVP + Artifacts Tasks ( X4, X6 and X8)

#----------------------------------------Session-1----------------------------------------

edf_path1 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X4-eeg.edf'.format(P_Num, P_Num)
csv_path1 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Mark\\P{}-Ss1-X4-mrk.csv'.format(P_Num, P_Num)
epoch1 = process_file(edf_path1,csv_path1)
df1 = epoch1.to_data_frame()
df1.to_csv('P{}-Ss1-Data-X4.csv'.format(P_Num), index=False)
           
edf_path2 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X6-eeg.edf'.format(P_Num, P_Num)
csv_path2 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Mark\\P{}-Ss1-X6-mrk.csv'.format(P_Num, P_Num)
epoch2 = process_file(edf_path2,csv_path2)
df2 = epoch2.to_data_frame()
df2.to_csv('P{}-Ss1-Data-X6.csv'.format(P_Num), index=False)

edf_path3 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X8-eeg.edf'.format(P_Num, P_Num)
csv_path3 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Mark\\P{}-Ss1-X8-mrk.csv'.format(P_Num, P_Num)
epoch3 = process_file(edf_path3,csv_path3)
df3 = epoch3.to_data_frame()
df3.to_csv('P{}-Ss1-Data-X8.csv'.format(P_Num), index=False)

#----------------------------------------Session-2----------------------------------------

edf_path4 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X4-eeg.edf'.format(P_Num, P_Num)
csv_path4 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Mark\\P{}-Ss2-X4-mrk.csv'.format(P_Num, P_Num)
epoch4 = process_file(edf_path4,csv_path4)
df4 = epoch4.to_data_frame()
df4.to_csv('P{}-Ss2-Data-X4.csv'.format(P_Num), index=False)
           
edf_path5 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X6-eeg.edf'.format(P_Num, P_Num)
csv_path5 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Mark\\P{}-Ss2-X6-mrk.csv'.format(P_Num, P_Num)
epoch5 = process_file(edf_path5,csv_path5)
df5 = epoch5.to_data_frame()
df5.to_csv('P{}-Ss2-Data-X6.csv'.format(P_Num), index=False)

edf_path6 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X8-eeg.edf'.format(P_Num, P_Num)
csv_path6 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Mark\\P{}-Ss2-X8-mrk.csv'.format(P_Num, P_Num)
epoch6 = process_file(edf_path6,csv_path6)
df6 = epoch6.to_data_frame()
df6.to_csv('P{}-Ss2-Data-X8.csv'.format(P_Num), index=False)

#----------------------------------------Session-3----------------------------------------

edf_path7 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X4-eeg.edf'.format(P_Num, P_Num)
csv_path7 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Mark\\P{}-Ss3-X4-mrk.csv'.format(P_Num, P_Num)
epoch7 = process_file(edf_path7,csv_path7)
df7 = epoch7.to_data_frame()
df7.to_csv('P{}-Ss3-Data-X4.csv'.format(P_Num), index=False)
           
edf_path8 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X6-eeg.edf'.format(P_Num, P_Num)
csv_path8 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Mark\\P{}-Ss3-X6-mrk.csv'.format(P_Num, P_Num)
epoch8 = process_file(edf_path8,csv_path8)
df8 = epoch8.to_data_frame()
df8.to_csv('P{}-Ss3-Data-X6.csv'.format(P_Num), index=False)

edf_path9 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X8-eeg.edf'.format(P_Num, P_Num)
csv_path9 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Mark\\P{}-Ss3-X8-mrk.csv'.format(P_Num, P_Num)
epoch9 = process_file(edf_path9,csv_path9)
df9 = epoch9.to_data_frame()
df9.to_csv('P{}-Ss3-Data-X8.csv'.format(P_Num), index=False)

#----------------------------------------Session-4----------------------------------------

edf_path10 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X4-eeg.edf'.format(P_Num, P_Num)
csv_path10 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Mark\\P{}-Ss4-X4-mrk.csv'.format(P_Num, P_Num)
epoch10 = process_file(edf_path10,csv_path10)
df10 = epoch10.to_data_frame()
df10.to_csv('P{}-Ss4-Data-X4.csv'.format(P_Num), index=False)
           
edf_path11 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X6-eeg.edf'.format(P_Num, P_Num)
csv_path11 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Mark\\P{}-Ss4-X6-mrk.csv'.format(P_Num, P_Num)
epoch11 = process_file(edf_path11,csv_path11)
df11 = epoch11.to_data_frame()
df11.to_csv('P{}-Ss4-Data-X6.csv'.format(P_Num), index=False)

edf_path12 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X8-eeg.edf'.format(P_Num, P_Num)
csv_path12 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Mark\\P{}-Ss4-X8-mrk.csv'.format(P_Num, P_Num)
epoch12 = process_file(edf_path12,csv_path12)
df12 = epoch12.to_data_frame()
df12.to_csv('P{}-Ss4-Data-X8.csv'.format(P_Num), index=False)


Extracting EDF parameters from C:\Users\muhammad\Desktop\Data A\P1\Session-1\Data\P1-Ss1-X4-eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 110999  =      0.000 ...   110.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 30.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['EventCode 16  EpochE', 'Impedance']
Number of markers:  360
Not setting metadata
Not setting metadata
360 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 360 events and 126 original time points ...
0 bad epochs dropped


C:\Users\muhammad\AppData\Local\Temp\ipykernel_18668\2222423111.py:57: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epoch = mne.Epochs(raw, new_events, tmin=-0.25, tmax=1.0, event_id=event_dict,


Extracting EDF parameters from C:\Users\muhammad\Desktop\Data A\P1\Session-1\Data\P1-Ss1-X6-eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 107999  =      0.000 ...   107.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 30.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['EventCode 16  EpochE', 'Impedance']
Number of markers:  360
Not setting metadata
Not setting metadata
360 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 360 events and 126 original time points ...
0 bad epochs dropped


C:\Users\muhammad\AppData\Local\Temp\ipykernel_18668\2222423111.py:57: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epoch = mne.Epochs(raw, new_events, tmin=-0.25, tmax=1.0, event_id=event_dict,


Extracting EDF parameters from C:\Users\muhammad\Desktop\Data A\P1\Session-1\Data\P1-Ss1-X8-eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 111999  =      0.000 ...   111.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 30.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['EventCode 16  EpochE', 'Impedance']
Number of markers:  360
Not setting metadata
Not setting metadata
360 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 360 events and 126 original time points ...
0 bad epochs dropped


...

C:\Users\muhammad\AppData\Local\Temp\ipykernel_18668\2222423111.py:57: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epoch = mne.Epochs(raw, new_events, tmin=-0.25, tmax=1.0, event_id=event_dict,


Extracting EDF parameters from C:\Users\muhammad\Desktop\Data A\P1\Session-4\Data\P1-Ss4-X6-eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 106999  =      0.000 ...   106.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 30.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['EventCode 16  EpochE', 'Impedance']
Number of markers:  360
Not setting metadata
Not setting metadata
360 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 360 events and 126 original time points ...
0 bad epochs dropped


C:\Users\muhammad\AppData\Local\Temp\ipykernel_18668\2222423111.py:57: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epoch = mne.Epochs(raw, new_events, tmin=-0.25, tmax=1.0, event_id=event_dict,


Extracting EDF parameters from C:\Users\muhammad\Desktop\Data A\P1\Session-4\Data\P1-Ss4-X8-eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 103999  =      0.000 ...   103.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 30.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['EventCode 16  EpochE', 'Impedance']
Number of markers:  360
Not setting metadata
Not setting metadata
360 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 360 events and 126 original time points ...
0 bad epochs dropped


C:\Users\muhammad\AppData\Local\Temp\ipykernel_18668\2222423111.py:57: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epoch = mne.Epochs(raw, new_events, tmin=-0.25, tmax=1.0, event_id=event_dict,


In [6]:
# Only Artifacts Tasks ( X3, X5 and X7)

#----------------------------------------Session-1----------------------------------------

edf_path1 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X3-eeg.edf'.format(P_Num, P_Num)
epoch1 = process_file2(edf_path1)
df1 = epoch1.to_data_frame()
df1.to_csv('P{}-Ss1-Data-X3.csv'.format(P_Num), index=False)
           
edf_path2 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X5-eeg.edf'.format(P_Num, P_Num)
epoch2 = process_file2(edf_path2)
df2 = epoch2.to_data_frame()
df2.to_csv('P{}-Ss1-Data-X5.csv'.format(P_Num), index=False)

edf_path3 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X7-eeg.edf'.format(P_Num, P_Num)
epoch3 = process_file2(edf_path3)
df3 = epoch3.to_data_frame()
df3.to_csv('P{}-Ss1-Data-X7.csv'.format(P_Num), index=False)

#----------------------------------------Session-2----------------------------------------

edf_path4 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X3-eeg.edf'.format(P_Num, P_Num)
epoch4 = process_file2(edf_path4)
df4 = epoch4.to_data_frame()
df4.to_csv('P{}-Ss2-Data-X3.csv'.format(P_Num), index=False)
           
edf_path5 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X5-eeg.edf'.format(P_Num, P_Num)
epoch5 = process_file2(edf_path5)
df5 = epoch5.to_data_frame()
df5.to_csv('P{}-Ss2-Data-X5.csv'.format(P_Num), index=False)

edf_path6 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X7-eeg.edf'.format(P_Num, P_Num)
epoch6 = process_file2(edf_path6)
df6 = epoch6.to_data_frame()
df6.to_csv('P{}-Ss2-Data-X7.csv'.format(P_Num), index=False)

#----------------------------------------Session-3----------------------------------------

edf_path7 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X3-eeg.edf'.format(P_Num, P_Num)
epoch7 = process_file2(edf_path7)
df7 = epoch7.to_data_frame()
df7.to_csv('P{}-Ss3-Data-X3.csv'.format(P_Num), index=False)
           
edf_path8 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X5-eeg.edf'.format(P_Num, P_Num)
epoch8 = process_file2(edf_path8)
df8 = epoch8.to_data_frame()
df8.to_csv('P{}-Ss3-Data-X5.csv'.format(P_Num), index=False)

edf_path9 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X7-eeg.edf'.format(P_Num, P_Num)
epoch9 = process_file2(edf_path9)
df9 = epoch9.to_data_frame()
df9.to_csv('P{}-Ss3-Data-X7.csv'.format(P_Num), index=False)

#----------------------------------------Session-4----------------------------------------

edf_path10 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X3-eeg.edf'.format(P_Num, P_Num)
epoch10 = process_file2(edf_path10)
df10 = epoch10.to_data_frame()
df10.to_csv('P{}-Ss4-Data-X3.csv'.format(P_Num), index=False)
           
edf_path11 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X5-eeg.edf'.format(P_Num, P_Num)
epoch11 = process_file2(edf_path11)
df11 = epoch11.to_data_frame()
df11.to_csv('P{}-Ss4-Data-X5.csv'.format(P_Num), index=False)

edf_path12 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X7-eeg.edf'.format(P_Num, P_Num)
epoch12 = process_file2(edf_path12)
df12 = epoch12.to_data_frame()
df12.to_csv('P{}-Ss4-Data-X7.csv'.format(P_Num), index=False)


Extracting EDF parameters from C:\Users\muhammad\Desktop\Data A\P1\Session-1\Data\P1-Ss1-X3-eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 108999  =      0.000 ...   108.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 30.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['EventCode 16  EpochE', 'Impedance']
Markers =  [ 1418 10423]
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 9001 original time points ...
0 bad epochs dropped
Extracting EDF parameters from

- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 30.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['EventCode 16  EpochE', 'Impedance']
Markers =  [ 3369 12375]
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 9001 original time points ...
0 bad epochs dropped
Extracting EDF parameters from C:\Users\muhammad\Desktop\Data A\P1\Session-4\Data\P1-Ss4-X3-eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 100999  =      0.000 ...   100.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, 

In [7]:
# Baselines ( B0, B1 and B2)

#----------------------------------------Session-1----------------------------------------

edf_path1 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-B0-eeg.edf'.format(P_Num, P_Num)
epoch1 = process_file2(edf_path1)
df1 = epoch1.to_data_frame()
df1.to_csv('P{}-Ss1-Data-B0.csv'.format(P_Num), index=False)
           
edf_path2 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-B1-eeg.edf'.format(P_Num, P_Num)
epoch2 = process_file2(edf_path2)
df2 = epoch2.to_data_frame()
df2.to_csv('P{}-Ss1-Data-B1.csv'.format(P_Num), index=False)

edf_path3 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-B2-eeg.edf'.format(P_Num, P_Num)
epoch3 = process_file2(edf_path3)
df3 = epoch3.to_data_frame()
df3.to_csv('P{}-Ss1-Data-B2.csv'.format(P_Num), index=False)

#----------------------------------------Session-2----------------------------------------

edf_path4 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-B0-eeg.edf'.format(P_Num, P_Num)
epoch4 = process_file2(edf_path4)
df4 = epoch4.to_data_frame()
df4.to_csv('P{}-Ss2-Data-B0.csv'.format(P_Num), index=False)
           
edf_path5 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-B1-eeg.edf'.format(P_Num, P_Num)
epoch5 = process_file2(edf_path5)
df5 = epoch5.to_data_frame()
df5.to_csv('P{}-Ss2-Data-B1.csv'.format(P_Num), index=False)

edf_path6 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-B2-eeg.edf'.format(P_Num, P_Num)
epoch6 = process_file2(edf_path6)
df6 = epoch6.to_data_frame()
df6.to_csv('P{}-Ss2-Data-B2.csv'.format(P_Num), index=False)

#----------------------------------------Session-3----------------------------------------

edf_path7 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-B0-eeg.edf'.format(P_Num, P_Num)
epoch7 = process_file2(edf_path7)
df7 = epoch7.to_data_frame()
df7.to_csv('P{}-Ss3-Data-B0.csv'.format(P_Num), index=False)
           
edf_path8 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-B1-eeg.edf'.format(P_Num, P_Num)
epoch8 = process_file2(edf_path8)
df8 = epoch8.to_data_frame()
df8.to_csv('P{}-Ss3-Data-B1.csv'.format(P_Num), index=False)

edf_path9 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-B2-eeg.edf'.format(P_Num, P_Num)
epoch9 = process_file2(edf_path9)
df9 = epoch9.to_data_frame()
df9.to_csv('P{}-Ss3-Data-B2.csv'.format(P_Num), index=False)

#----------------------------------------Session-4----------------------------------------

edf_path10 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-B0-eeg.edf'.format(P_Num, P_Num)
epoch10 = process_file2(edf_path10)
df10 = epoch10.to_data_frame()
df10.to_csv('P{}-Ss4-Data-B0.csv'.format(P_Num), index=False)
           
edf_path11 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-B1-eeg.edf'.format(P_Num, P_Num)
epoch11 = process_file2(edf_path11)
df11 = epoch11.to_data_frame()
df11.to_csv('P{}-Ss4-Data-B1.csv'.format(P_Num), index=False)

edf_path12 = r'C:\\Users\\muhammad\\Desktop\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-B2-eeg.edf'.format(P_Num, P_Num)
epoch12 = process_file2(edf_path12)
df12 = epoch12.to_data_frame()
df12.to_csv('P{}-Ss4-Data-B2.csv'.format(P_Num), index=False)


Extracting EDF parameters from C:\Users\muhammad\Desktop\Data A\P1\Session-1\Data\P1-Ss1-B0-eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    30.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 30.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['EventCode 16  EpochE', 'Impedance']
Markers =  [1332 2335]
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 1001 original time points ...
0 bad epochs dropped
Extracting EDF parameters from C:

- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 30.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['EventCode 16  EpochE', 'Impedance']
Markers =  [ 830 6833]
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 6001 original time points ...
0 bad epochs dropped
Extracting EDF parameters from C:\Users\muhammad\Desktop\Data A\P1\Session-4\Data\P1-Ss4-B0-eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    30.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, aft